# Aviation Risk Analysis: Identifying the Safest Aircraft Models

## Project Overview
The company is exploring a new business venture: purchasing and operating aircraft for both commercial and private services. Before making such a significant investment, leadership needs to understand which aircraft models present the lowest operational risk. This project uses historical accident data from the U.S. National Transportation Safety Board (NTSB) to identify the safest aircraft types and provide clear, evidence-based recommendations. The analysis focuses on real-world aviation accident records from 1962 through 2023.

## Business Understanding
The key stakeholder is the head of the new Aviation Division. Their central question is: **Which aircraft models are the safest to acquire and operate?** To answer this, we will clean and explore the NTSB accident dataset, calculate risk metrics such as fatal- or serious-injury rates, and evaluate how different aircraft characteristics, flight conditions, and operational factors contribute to accident severity. The end result will be a set of actionable insights, backed by data, that guide purchase decisions and reduce safety risks.

## Project Objectives
* Identify the aircraft makes and models with the lowest accident and fatal-injury rates in the NTSB dataset from 1962–2023.
* Analyze how factors such as number of engines, flight purpose, weather conditions, and phase of flight affect accident severity.
* Provide clear, data-backed recommendations to guide the company’s aircraft purchase and operational decisions.

### 1. Data Acquisition & Loading

In [14]:
# Import Liabraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
# Load Dataset
df = pd.read_csv('./NTSB_database.csv')
df.head()

,Event Id,Investigation Type,Country,Aircraft Damage,Aircraft Category,Make,Model,Amateur Built,Number Of Engines,Engine Type,...,Event Year,Publication Year,Event Month,Publication Month,Event Day,Publication Day,Date Difference,Publication Month Name,Event Month Name,Season
0,20001218X45444,Accident,United States,Destroyed,fixed wing single engine,stinson,108-3,No,1,reciprocating,...,1948,2001,10,8,24,24.0,26,August,October,Fall
1,20001218X45447,Accident,United States,Destroyed,weight-shift-control,piper,pa24-180,No,1,reciprocating,...,1962,1996,7,9,19,19.0,34,September,July,Summer
2,20061025X01555,Accident,United States,Destroyed,fixed wing single engine,cessna,172m,No,1,reciprocating,...,1974,2007,8,2,30,30.0,33,February,August,Summer
3,20001218X45448,Accident,United States,Destroyed,weight-shift-control,rockwell,112,No,1,reciprocating,...,1977,2000,6,12,19,19.0,23,December,June,Summer
4,20041105X01764,Accident,United States,Destroyed,fixed wing multi engine,cessna,501,No,2,turbo fan,...,1979,1980,8,4,2,2.0,1,April,August,Summer


In [16]:
# Initial Checks
df.shape

(87951, 45)

In [17]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87951 entries, 0 to 87950
Data columns (total 45 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Event Id                      87951 non-null  object 
 1   Investigation Type            87951 non-null  object 
 2   Country                       87951 non-null  object 
 3   Aircraft Damage               87951 non-null  object 
 4   Aircraft Category             87951 non-null  object 
 5   Make                          87951 non-null  object 
 6   Model                         87951 non-null  object 
 7   Amateur Built                 87951 non-null  object 
 8   Number Of Engines             87951 non-null  int64  
 9   Engine Type                   87951 non-null  object 
 10  Far Description               87951 non-null  object 
 11  Schedule                      87951 non-null  object 
 12  Purpose Of Flight             87951 non-null  object 
 13  T

In [22]:
df.describe()

,Number Of Engines,Total Fatal Injuries,Total Serious Injuries,Total Minor Injuries,Total Uninjured,Longitude,Latitude,Number Of Seats,Type Aircraft,Type Engine,...,Purpose Of Flight Factorized,Make Factorized,Model Factorized,Event Year,Publication Year,Event Month,Publication Month,Event Day,Publication Day,Date Difference
count,87951.000000,87951.000000,87951.000000,87951.000000,87951.000000,87951.000000,87951.000000,87951.000000,87951.000000,87951.000000,...,87951.000000,87951.000000,87951.000000,87951.000000,87951.000000,87951.000000,87951.000000,87951.000000,87951.000000,87951.000000
mean,1.143171,0.534184,0.276881,0.316290,4.325181,-84.438795,37.456033,11.055565,7.491592,1.320224,...,1.850678,600.506009,2101.362918,1999.265455,2002.113347,6.589385,6.903799,15.728860,15.728860,2.886766
std,0.450370,3.075452,1.336561,1.793138,22.312659,46.966700,14.067686,40.533493,3.560943,0.977549,...,3.094395,1507.969996,2970.271576,11.903151,13.011296,3.061984,3.031236,8.839005,8.839005,3.544021
min,0.000000,0.000000,0.000000,0.000000,0.000000,-177.375430,-77.848335,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,1948.000000,1980.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,-110.929451,33.370320,3.000000,4.000000,1.000000,...,0.000000,2.000000,111.000000,1989.000000,1992.000000,4.000000,4.000000,8.000000,8.000000,1.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000,-90.664799,38.729625,4.000000,7.000000,1.000000,...,0.000000,6.000000,442.000000,1998.000000,2000.000000,7.000000,8.000000,16.000000,16.000000,1.000000
75%,1.000000,0.000000,0.000000,0.000000,2.000000,-79.384030,43.207178,5.000000,12.000000,1.000000,...,3.000000,59.000000,3324.000000,2009.000000,2020.000000,9.000000,9.000000,23.000000,23.000000,3.000000
max,8.000000,270.000000,137.000000,380.000000,459.000000,179.559745,90.000000,660.000000,21.000000,14.000000,...,25.000000,7551.000000,11562.000000,2022.000000,2022.000000,12.000000,12.000000,31.000000,31.000000,38.000000


### Data Understanding

We’ll keep only the fields needed to filter by time and measure safety severity by make/model. These allows us clean the data, slice recent years (2010–2022) and compute severity rates that drive buy or no-buy recommendations.


* *Event Id* — unique identifier (for de-duplication and joins).
* *Event Year / Month / Day* — when it happened (time filters & trends).
* *Make* — manufacturer (compare brands).
* *Model* — specific aircraft type (model-level rankings).
* *Aircraft Category* — aircraft class (compare like-with-like).
* *Number Of Engines* — single vs multi (risk differs).
* *Engine Type* — piston / turboprop / jet / other (risk differs).
* *Aircraft Damage* — rough severity proxy (minor/substantial/destroyed).
* *Broad Phase Of Flight* — takeoff/landing/etc. (phase risk).
* *Weather Condition* — VMC/IMC etc. (weather-related risk).
* *Purpose Of Flight* *(if present)* — personal/training/commercial (ops context).
* *Injury columns* — *Total Fatal / Serious / Minor / Uninjured* (define severity).


In [20]:
# Data quality check
# Check for uniqueness in the Event ID column
df['Event Id'].duplicated().sum()

0

In [21]:
# Ensure the event date fields exist so that we can filter by time
'Event Year' in df.columns, 'Event Month' in df.columns, 'Event Day' in df.columns

(True, True, True)